In [2]:
import pandas as pandas

In [3]:
def tasks_from_rishi(spreadsheet_path, resources_dict=None,
                           sheetname='tasks', resources_sheetname='resources',
                           tasks_color="blue",
                           task_name_prefix="", sep=";"):
    if resources_dict is None:
        resources_dict = resources_from_spreadsheet(
            spreadsheet_path, sheetname=resources_sheetname)


    if spreadsheet_path.endswith("csv"):
        process_df = pandas.read_csv(spreadsheet_path, sep=sep)
    else:
        process_df = pandas.read_excel(spreadsheet_path,
                                       sheet_name=sheetname)
    process_tasks = {}
    tasks_list = []
    for i, row in process_df.iterrows():
        task_resources = [
            resources_dict[r.strip()]
            for r in row.resources.split(",")
        ]
        follows = str(row.follows)
        if follows == "nan":
            follows = ()
        else:
            follows = [process_tasks[t.strip()] for t in follows.split(",")]

        if str(row.scheduled_resources) == "nan":
            scheduled_resources = None
        else:
            scheduled_resources = {
                resources_dict[r.split(":")[0].strip()]: int(r.split(":")[1])
                for r in row.scheduled_resources.split(",")
            }
        new_task = Task(
            name=task_name_prefix + row.task,
            resources=task_resources,
            duration=row.duration,
            follows=follows,
            color=(tasks_color if (str(row.color) == "nan")
                               else row.color),
            max_wait=(None if (str(row.max_wait) == "nan")
                           else int(row.max_wait)),
            scheduled_start=(None if (str(row.scheduled_start) == "nan")
                                  else int(row.scheduled_start)),
            scheduled_resources=scheduled_resources
        )
        process_tasks[row.task] = new_task
        tasks_list.append(new_task)

    return tasks_list

In [4]:
%matplotlib qt

In [5]:
from taskpacker import (tasks_from_spreadsheet,
                        resources_from_spreadsheet,
                        schedule_processes_series,
                        plot_tasks_dependency_graph,
                        plot_schedule, Task)
import os
import matplotlib.cm as cm

ALLOW_BREAKS = False
NUM_SAMPLES = 60

colors = (cm.Spectral(0.21 * i % 1.0) for i in range(NUM_SAMPLES))


# LOAD THE PROCESS AND RESOURCES CONFIGURATIONS

spreadsheet_path = os.path.join("dna_assembly.xls")

resources = resources_from_spreadsheet(spreadsheet_path=spreadsheet_path,
                                       sheetname="resources")

processes = [
    tasks_from_rishi(spreadsheet_path=spreadsheet_path,
                           sheetname="process",
                           resources_dict=resources,
                           tasks_color=next(colors),
                           task_name_prefix="WU%d_" % (i + 1))
    for i in range(NUM_SAMPLES)
]


# CREATE THE BREAKS

# if ALLOW_BREAKS:
#     scheduled_breaks = [
#         Task("break_%03d" % i,
#              resources=[resources["igor"]],
#              scheduled_resources={resources["igor"]: 1},
#              duration=12 * 60, scheduled_start=24 * 60 * i,
#              color='white')
#         for i in range(6)
#     ]
# else:
#     scheduled_breaks = []


# # OPTIMIZE THE SCHEDULE

# print("NOW OPTIMIZING THE SCHEDULE, BE PATIENT...")
# new_processes = schedule_processes_series(
#     processes, est_process_duration=5000, time_limit=5,
#     scheduled_tasks=scheduled_breaks
# )


# # PLOT THE TASKS DEPENDENCY TREE

# ax = plot_tasks_dependency_graph(processes[0])
# ax.set_title("PLAN OF A PROCESS")
# ax.figure.savefig("dna_assembly_process.pdf", bbox_inches="tight")


# # PLOT THE OPTIMIZED SCHEDULE

# all_tasks = [t for process in new_processes for t in process] + scheduled_breaks
# ax = plot_schedule(all_tasks)
# # ax.set_xlim((5800, 6500))
# ax.figure.set_size_inches((10, 5))
# ax.set_xlabel("time (seconds)")
# # plt.savefig('hp_limited')


In [22]:
resources

{'gantry': gantry, 'hp': hp, 'ot2': ot2}

In [24]:
item = resources['gantry']
props = [p for p in dir(item) if '_' not in p]
for p in props:print(f'{p}:{getattr(item,p)}')

capacity:1
hash:<bound method Resource.hash of gantry>
name:gantry


In [14]:
processes[0][4].max_wait

30

In [15]:
all_tasks = processes

In [18]:
props = [p for p in dir(all_tasks[0][1]) if '_' not in p]

In [19]:
for p in props:print(f'{p}:{getattr(all_tasks[0][1],p)}')

color:(0.6196078431372549, 0.00392156862745098, 0.25882352941176473, 1.0)
duration:60
follows:[Task(WU1_move_storage_ot2, 10)]
id:WU1_spincoat176708970027854230177174200101110379380
name:WU1_spincoat
priority:1
resources:[ot2]
